In [3]:
import pykitti
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# 坐标转换

In [4]:
basedir = '/media/cyj/DATA/hybrid_feature_LO/datasets'
date = '2011_10_03'
drive = '0027'
dataset = pykitti.raw(basedir, date, drive)
data = dataset.get_velo(0)
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(data[:, :3])

绕坐标轴旋转

In [5]:
from scipy.spatial.transform import Rotation as R
R1 = R.from_euler('y', 90, degrees=True).as_matrix()
R2 = R.from_euler('x', -90, degrees=True).as_matrix()
o3d.visualization.draw_geometries([point_cloud.rotate(R1, center=point_cloud.get_center()).rotate(R2, center=point_cloud.get_center())])
# 复合旋转
R3 = R.from_euler('zyx', [45, 30, 60], degrees=True).as_matrix()
o3d.visualization.draw_geometries([point_cloud.rotate(R3, center=point_cloud.get_center())])

使用四元数、旋转向量旋转

In [12]:
R4 = R.from_quat([0.5, 0.5, 0.5, 0.5])
data4 = np.dot(data[:, :3], R4.as_matrix())
point_cloud4 = o3d.geometry.PointCloud()
point_cloud4.points = o3d.utility.Vector3dVector(data4)
o3d.visualization.draw_geometries([point_cloud4])

R5 = R.from_rotvec([0.1, 0.2, 0.3])
data5 = np.dot(data[:, :3], R5.as_matrix())
point_cloud5 = o3d.geometry.PointCloud()
point_cloud5.points = o3d.utility.Vector3dVector(data5)
o3d.visualization.draw_geometries([point_cloud5])

# 将GT坐标 from cam to velo

读取GT坐标、calib文件

In [3]:
def read_calib_file(filepath):
    calib_data = {}
    with open(filepath, 'r') as file:
        for line in file:
            key, value = line.split(':', 1)
            if key == 'R' or key == 'T':
                calib_data[key] = value.strip()

    return calib_data


filepath = './datasets/2011_10_03/calib_velo_to_cam.txt'
calib_data = read_calib_file(filepath)

# 解析和转换 R 和 T 数据
if "R" in calib_data and "T" in calib_data:
    R_values = [float(x) for x in calib_data["R"].split()]
    T_values = [float(x) for x in calib_data["T"].split()]

    # 构造变换矩阵
    R_matrix = np.array(R_values).reshape(3, 3)
    T_vector = np.array(T_values).reshape(3, 1)
    RT_matrix = np.hstack((R_matrix, T_vector))
    RT_matrix = np.vstack((RT_matrix, [0, 0, 0, 1]))

    RT_matrix
else:
    RT_matrix = None

RT_matrix



array([[ 7.967514e-03, -9.999679e-01, -8.462264e-04, -1.377769e-02],
       [-2.771053e-03,  8.241710e-04, -9.999958e-01, -5.542117e-02],
       [ 9.999644e-01,  7.969825e-03, -2.764397e-03, -2.918589e-01],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  1.000000e+00]])

对GT坐标进行变换

In [12]:
RT_matrix_inv = np.linalg.inv(RT_matrix)

gt_filepath = './datasets/2011_10_03/2011_10_03_drive_0027_sync/00.txt'

# 读取轨迹数据
with open(gt_filepath, 'r') as f:
    lines = f.readlines()

transformed_lines = []

for line in lines:
    pose = line.strip().split()  # 切分每行数据

     # 提取位姿变换矩阵并变为4x4
    pose_matrix = np.eye(4)
    
    pose_matrix[:3, :] = np.array(pose).reshape(3, 4).astype(float)

    # 旋转矩阵
    transformed_matrix = RT_matrix_inv @ pose_matrix @ RT_matrix

    # 从变换矩阵中提取变换后的位姿
    transformed_pose = transformed_matrix[:3, :].flatten()
    transformed_lines.append(" ".join(map(str, transformed_pose)))

# 保存变换后的轨迹
save_filepath = './datasets/2011_10_03/2011_10_03_drive_0027_sync/groundtruth.txt'

with open(save_filepath, 'w') as f:
    f.write("\n".join(transformed_lines))


In [1]:
import numpy as np
import pykitti
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
# 读取数据集
basedir = 'datasets'
date = '2011_10_03'
drive = '0027'
dataset = pykitti.raw(basedir, date, drive)
data = dataset.get_velo(5)
point_cloud1 = o3d.geometry.PointCloud()
point_cloud1.points = o3d.utility.Vector3dVector(data[:, :3])
data = dataset.get_velo(10)
point_cloud2 = o3d.geometry.PointCloud()
point_cloud2.points = o3d.utility.Vector3dVector(data[:, :3])

In [ ]:

# 使用真实的变换矩阵变换点云
draw_registration_result(point_cloud1, point_cloud2, transformed_matrix)\

#初始的cam是world
#GT velo to world

## GT velo to velo

#ICP velo to velo

#draw velo to velo